# Day 5

## Part 1

In [13]:
input = open('./data/input_day5.txt').read().splitlines()
input[:5]

['seeds: 763445965 78570222 1693788857 146680070 1157620425 535920936 3187993807 180072493 1047354752 20193861 2130924847 274042257 20816377 596708258 950268560 11451287 3503767450 182465951 3760349291 265669041',
 '',
 'seed-to-soil map:',
 '0 1894195346 315486903',
 '1184603419 2977305241 40929361']

In [14]:
sample = open('./data/sample_day5.txt').read().splitlines()
sample[:5]

['seeds: 79 14 55 13', '', 'seed-to-soil map:', '50 98 2', '52 50 48']

In [15]:
import re
from collections import defaultdict
from functools import partial

def parse_seeds(seed: str) -> list[int]:
    return [int(i) for i in re.findall(r'\d+', seed)]


def parse_ranges(line: str) -> dict[int, int]:
    destination, source, range_ = map(int, line.split(" "))
    return partial(range_function, source=source, destination=destination, range_=range_)


def range_function(input: int, source: int, destination:int, range_:int) -> int:
    if input >= source and input < (source + range_):
        return destination + input - source
    return input


def parse_maps(maps: list[str]) -> dict[str, dict[int, int]]:
    almanac = defaultdict(list)
    map_key = None
    for line in maps:
        if 'map' in line:
            map_key = line.replace("map:", "").strip()
        elif line != "":
            almanac[map_key].append(parse_ranges(line))
    return almanac


def curry(input: int, functions: list[partial]) -> int:
    for function in functions:
        output = function(input)
        if input != output:
            return output
    return input


def day05_part1(input):
    seeds = parse_seeds(input[0])
    almanac = parse_maps(input[2:])
    locations = []
    for seed in seeds:
        soil = curry(seed, almanac["seed-to-soil"])
        fertilizer = curry(soil, almanac["soil-to-fertilizer"])
        water = curry(fertilizer, almanac["fertilizer-to-water"])
        light = curry(water, almanac["water-to-light"])
        temperature = curry(light, almanac["light-to-temperature"])
        humidity = curry(temperature, almanac["temperature-to-humidity"])
        location = curry(humidity, almanac["humidity-to-location"])
        locations.append(location)
    return min(locations)


assert day05_part1(sample) == 35, day05_part1(sample)

day05_part1(input)

265018614

## Part 2

In [22]:

def parse_seeds(seed: str) -> list[int]:
    return [int(i) for i in re.findall(r'\d+', seed)]


def range_function(input: tuple[int, int], seeds: list[tuple[int, int]], destination:int, source: int, source_range:int) -> int:
    seed, seed_range = input
    max_tail = max(source, seed)
    min_head = min(source + source_range, seed + seed_range)
    if max_tail < min_head:
        if max_tail > seed:
            seeds.append((seed, max_tail - seed))
        if (seed + seed_range) > min_head:
            seeds.append((min_head, seed + seed_range - min_head))
        return destination + max_tail - source, min_head - max_tail
    return input


def parse_ranges(line: str) -> dict[int, int]:
    destination, source, source_range = map(int, line.split(" "))
    return partial(range_function, destination=destination, source=source, source_range=source_range)


def parse_maps(maps: list[str]) -> dict[str, dict[int, int]]:
    almanac = defaultdict(list)
    map_key = None
    for line in maps:
        if 'map' in line:
            map_key = line.replace("map:", "").strip()
        elif line != "":
            almanac[map_key].append(parse_ranges(line))
    return almanac


def curry(input: tuple[int, int], seeds: list[tuple[int, int]], functions: list[partial]) -> int:
    for function in functions:
        output = function(input, seeds)
        if input != output:
            return output
    return input


def day05_part2(input):
    seeds = parse_seeds(input[0])
    almanac = parse_maps(input[2:])
    locations = []
    seeds = [(seeds[i], seeds[i+1]) for i in range(0, len(seeds), 2)]
    blocks = (
        almanac["seed-to-soil"],
        almanac["soil-to-fertilizer"],
        almanac["fertilizer-to-water"],
        almanac["water-to-light"],
        almanac["light-to-temperature"],
        almanac["temperature-to-humidity"],
        almanac["humidity-to-location"],
    )
    for block in blocks:
        new = []
        while len(seeds) > 0:
            seed = seeds.pop(0)
            new.append(curry(seed, seeds, block))
        seeds = new
            
    #return locations
    return min(seeds)[0]


assert day05_part2(sample) == 46, day05_part2(sample)

day05_part2(input)

63179500

# Day 6

## Part 1

In [ ]:
times, distances = open('data/input_day6.txt').read().splitlines()
times = list(map(int, times.split(":")[1].split()))
distances = list(map(int, distances.split(":")[1].split()))

def day06_part1(input: list[tuple[int, int]]) -> int:
    winners = 1
    for time, distance in input:
        #print(time, distance)
        reaches = [s * (time - s) for s in range(time + 1)]
        #print(reaches)
        winners *= len([r for r in reaches if r > distance])
    return winners

day06_part1(zip(times, distances))

449550

In [ ]:
time, distance = open('data/input_day6.txt').read().splitlines()
time = int(time.split(":")[1].replace(" ", ""))
distance = int(distance.split(":")[1].replace(" ", ""))

def day06_part1(input: tuple[int, int]) -> int:
    time, distance = input
    current_speed = distance // time
    print(current_speed)
    for t in range(current_speed, distance, 1):
        if t * (time - t) > distance: 
            min_speed_to_win = t
            break
    print(min_speed_to_win)
    for t in range(time, -1, -1):
        if t * (time - t) > distance: 
            max_speed_to_win = t
            break
    print(max_speed_to_win)
    winners = max_speed_to_win - min_speed_to_win + 1
    return winners

day06_part1((time, distance))

7413271
9234170
37594309


28360140

# Day 7

# Part 1

In [ ]:
data = open('data/input_day7.txt').read().splitlines()
data = [line.split(" ") for line in data]
hands, bids = zip(*data)
bids = list(map(int, bids))

In [ ]:
from collections import defaultdict

def joker_card(hand: str) -> str:
    if "J" not in hand: 
        return hand
    count_cards = {c: 0 for c in set(hand)}
    for card in hand:
        count_cards[card] += 1
    js = count_cards["J"]
    del count_cards["J"]
    if js == 5:
        return hand 
    if js < 3:
        return hand.replace("J", max(count_cards, key=count_cards.get))
    return hand.replace("J", min(count_cards, key=count_cards.get))


def rank_type(hand: str) -> int:
    hand = joker_card(hand)
    cards = set(hand)
    if len(cards) == 1:
        return 6
    if len(cards) == 5:
        return 0
    
    count_cards = {c: 0 for c in cards}
    for card in hand:
        count_cards[card] += 1
    
    count = set(count_cards.values())
   
    if count == set([4, 1]):      
        return 5
    if count == set([3, 2]):
        return 4
    if count == set([3, 1]):
        return 3
    if count == set([2, 1]) and len(cards) == 3:
        return 2
    return 1


def card_label(card: str, bid: int) -> int:
    l = 'J23456789TQKA'
    return [l.index(c) for c in card] + [card, bid]


def day07_part1(hands: list[str], bids: list[int]) -> str:
    winners = 0
    types = []
    for h, b in zip(hands, bids):
        types.append((rank_type(h), *card_label(h, b)))
    
    sorted_types = sorted(types)
    
    for i, *card in enumerate(sorted_types):
        winners += (i + 1) * card[0][-1]
    
    return winners


day07_part1(hands, bids)

250382098

# Day 8

## Parte 1

In [1]:
import re

instructions, maps = open('./data/sample_day8.txt').read().split("\n\n")
maps = maps.splitlines()
maps = [re.findall(r"[A-Z0-9]+", line) for line in maps]
maps = {it[0]: (it[1], it[2]) for it in maps}
instructions

'LR'

In [34]:
def day08_part1(instructions: str, maps: list[list[str]]) -> str:
    size = len(instructions)
    i = 0
    c = 0
    lookup = 'AAA'
    while True:
        instruction = instructions[i]
        coord = maps[lookup]
        lookup = coord[1] if instruction == 'R' else coord[0]
        c += 1
        if lookup == 'ZZZ':
            break 
        i += 1
        if i > size - 1:
            i = 0
        
    return c

day08_part1(instructions, maps)

15871

## Part 2

In [6]:
import re

instructions, maps = open('./data/input_day8.txt').read().split("\n\n")
maps = maps.splitlines()
maps = [re.findall(r"[A-Z0-9]+", line) for line in maps]
maps = {it[0]: (it[1], it[2]) for it in maps}
instructions

'LLLLLLLRRRLRRRLRLRLRLRRLLRRRLRLLRRRLLRRLRRLRRLRLRRLRLRRRLRRLRLRRRLRRLRRLRLRRLLRLLRLRRRLRRLRLLLLRRLLLLRLRLRLRRRLRLRLLLRLRRRLRRRLRRRLRLRRLRRRLRLLLRLLRRLRRRLRRLRRLRRLRLRRRLRLRLRLLRRRLRRRLRRLRRRLLLRRLRRLRRRLRLRRRLRRRLRLRRLRRRLRLRRLRLRRLRRRLRLRRLRLLRRRLLRLRRLRRRLLLRLRRLRRRR'

In [64]:
def mdc(a: int, b: int) -> int:
    while b != 0:
        a, b = b, a % b
    return a

def mmc(a: int, b: int) -> int:
    return a * b // mdc(a, b)

def mmc_lista(numeros: list[int]) -> int:
    resultado = numeros[0]
    for numero in numeros[1:]:
        resultado = mmc(resultado, numero)
    return resultado

def day08_part2(instructions: str, maps: list[list[str]]) -> str:
    size = len(instructions)
    i = 0
    c = 1
    nodes = [n for n in maps if n.endswith("A")]
    cycles = {nb: 0 for nb in range(len(nodes))}
    blacklist = []
    while True:
        new_nodes = []
        instruction = instructions[i]
        for nb, node in enumerate(nodes): 
            coord = maps[node]
            lookup = coord[1] if instruction == 'R' else coord[0]
            if lookup.endswith("Z"):
                if cycles[nb] == 0:
                    cycles[nb] = c
                elif nb not in blacklist:
                    cycles[nb] = c - cycles[nb]
                    blacklist.append(nb)
            new_nodes.append(lookup)
        c += 1
        if all([v > 0 for v in cycles.values()]) and len(blacklist) == len(nodes):
            break 
        i += 1
        if i > size - 1:
            i = 0
        nodes = new_nodes

    return mmc_lista(list(cycles.values()))

day08_part2(instructions, maps)

11283670395017